## Problem Statement
A factory produces two products, P1 and P2. The profit per unit is:<br>
* P1: $40
* P2: $50

Machine time available:<br>
* Machine A: 100 hours
* Machine B: 80 hours
* Machine C: 90 hours

Time required per unit:
* P1- 1 hour on A, 2 hour on B
* P2- 2 hour on A, 1 hour on B, 3 hour on C

Objective: Maximize profit.

Decision variables:
* x₁: Units of P1 to produce.
* x₂: Units of P2 to produce.


# Using pyomo

1. Linear programming (LP): LP involves optimizing a linear objective function subject to linear equality and inequality constraints. It's widely used for resource allocation, scheduling, and financial planning problems.
2. Nonlinear programming (NLP): NLP deals with optimizing a nonlinear objective function with nonlinear constraints. It’s often used in engineering and economics.
3. Mixed-integer programming (MIP): MIP involves optimization problems where some variables are restricted to be integers while others can be continuous.
4. Stochastic programming: Stochastic programming addresses optimization problems where some elements are uncertain and modeled as random variables. It's commonly applied in finance and supply chain management to optimize decisions under uncertainty.
5. Dynamic optimization: Dynamic optimization focuses on optimizing decision variables over time, typically involving dynamically evolving systems. It is used in fields like process control, robotics, and economics to manage time-dependent processes.

## Installation
pip install pyomo<br>
1. GLPK (GNU Linear Programming Kit)- ```sudo apt-get install glpk-utils```
2. CBC (Coin-or Branch and Cut)- ```conda install -c conda-forge coincbc```
3. IPOPT (Interior Point OPTimizer)- ```conda install -c conda-forge ipopt```

## Defining Variables

1. Scalar variables<br>
We first import the Var and create a variable x using Var(). This variable has no specified bounds, meaning it can take any real value unless constrained otherwise in the model.
```
from pyomo.environ import Var
model.x = Var()
```

2. Adding bounds<br>
You can restrict the values that a variable can take by specifying bounds. Bounds are defined as a tuple (lower_bound, upper_bound):
```
from pyomo.environ import Var
model.x = Var(bounds=(0, None))
```

3. Specifying domains<br>
Pyomo provides predefined domains that you can use to specify the type of values a variable can take, such as NonNegativeReals, Integers, or Binary: 
```
from pyomo.environ import Var, NonNegativeReals
model.x = Var(domain=NonNegativeReals)
```

4. Indexed variables
When dealing with multiple variables that are similar in nature, such as variables representing different time periods or items, it's efficient to use indexed variables. Indexed variables are variables that are defined over a set.
```
from pyomo.environ import Var, Set, NonNegativeReals
model.I = Set(initialize=[1, 2, 3])
model.y = Var(model.I, domain=NonNegativeReals)
```
Suppose you're modeling the production quantities for three products. You can define:
```
model.Products = Set(initialize=['A', 'B', 'C'])
model.production = Var(model.Products, domain=NonNegativeReals)
```
Now, model.production['A'], model.production['B'], and model.production['C'] represent the production quantities for products A, B, and C, respectively.

### Parameterizing models
Parameters are fixed values used in the model to represent known quantities or constants that do not change during the optimization process. 
```
from pyomo.environ import Param
model.p = Param(initialize=5)
```
The code above defines a parameter p in the Pyomo model using the Param class and initializes it with a fixed value of 5. The parameter p can now be used in the model to represent a constant value that does not change during the optimization process.

## 1. Linear optimization Solution

In [4]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals) # x1>=0
model.x2 = pyo.Var(within=pyo.NonNegativeReals) # x2>=0

# Define objective
model.obj = pyo.Objective(expr=40*model.x1 + 50*model.x2, sense=pyo.maximize)

# Define constraints
# Machine A capacity constraint: 1x1 + 2x2 <= 100
model.con1 = pyo.Constraint(expr=model.x1 + 2*model.x2 <= 100)
# Machine B capacity constraint: 2x1 + 1x2 <= 80
model.con2 = pyo.Constraint(expr=2*model.x1 + model.x2 <= 80)
# Machine C capacity constraint: 3x2 <= 90
model.con3 = pyo.Constraint(expr=3*model.x2 <= 90)

# Select solver
solver = pyo.SolverFactory('glpk')
# Solve the problem
result = solver.solve(model)

# Display results
print('Status:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)
print('Optimal x1:', pyo.value(model.x1))
print('Optimal x2:', pyo.value(model.x2))
print('Optimal Objective:', pyo.value(model.obj))

Status: ok
Termination Condition: optimal
Optimal x1: 25.0
Optimal x2: 30.0
Optimal Objective: 2500.0


## 2. Nonlinear optimization

In [1]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

# Define variables with lower bounds
model.x = pyo.Var(bounds=(0, None)) # x>=0
model.y = pyo.Var(bounds=(0, None)) # y>=0

# Objective function: minimize (x - 1)² + (y - 2)²
model.obj = pyo.Objective(expr=(model.x - 1)**2 + (model.y - 2)**2, sense=pyo.minimize)

# Constraint: x² + y² ≤ 4 (circle of radius 2)
model.circle = pyo.Constraint(expr=model.x**2 + model.y**2 <= 4)

solver = pyo.SolverFactory('ipopt')
result = solver.solve(model)
print('Optimal x:', pyo.value(model.x))
print('Optimal y:', pyo.value(model.y))
print('Minimum Z:', pyo.value(model.obj))

Optimal x: 0.8944271937881092
Optimal y: 1.7888543856950594
Minimum Z: 0.05572808785166419


## 3. Mixed-integer programming (MIP)

In [6]:
import pyomo.environ as pyo

# objective- A company must decide whether to open warehouses in locations A, B, and C. 
# The goal is to minimize the total cost, which includes fixed costs of opening warehouses 
# and transportation costs.
 
locations = ['A', 'B', 'C']
FixedCost = {'A': 1000, 'B': 1200, 'C': 1500} # fixed cost per warehouse location
TransportCost = {'A': 5, 'B': 4, 'C': 6} # transportation cost per product as per warehouse
Capacity = {'A': 100, 'B': 80, 'C': 90} # product capacity per warehouse
Demand = 150 # minimum goods to transport

model = pyo.ConcreteModel()

# Binary variable: 1 if warehouse is open, 0 otherwise
model.y = pyo.Var(locations, domain=pyo.Binary)

# Continuous variable: amount of goods transported
model.x = pyo.Var(locations, domain=pyo.NonNegativeReals)

# objective
model.cost = pyo.Objective(
    expr=sum(FixedCost[i]*model.y[i] + TransportCost[i]*model.x[i] for i in locations),
    sense=pyo.minimize
)

# Demand constraint
model.demand = pyo.Constraint(expr=sum(model.x[i] for i in locations) >= Demand)

# Capacity constraints
def capacity_rule(model, i):
    return model.x[i] <= Capacity[i] * model.y[i]
model.capacity = pyo.Constraint(locations, rule=capacity_rule)

solver = pyo.SolverFactory('cbc')
result = solver.solve(model)
for i in locations:
    print(f"Warehouse {i}: Open={pyo.value(model.y[i])}, Transported={pyo.value(model.x[i])}")
print('Minimum Total Cost:', pyo.value(model.cost))

Warehouse A: Open=1.0, Transported=70.0
Warehouse B: Open=1.0, Transported=80.0
Warehouse C: Open=0.0, Transported=0.0
Minimum Total Cost: 2870.0
